In [1]:
import numpy as np 
def load_data():
    #读取数据
    import json
    datafile = 'work\housing.data'
    data = np.fromfile(datafile,sep = ' ')
    #把数据转换成每行14个元素的矩阵
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 
                     'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)
    data = data.reshape([data.shape[0] // feature_num, feature_num])
    #设置训练数据和测试数据
    ratio = 0.8
    offset = int(data.shape[0]*ratio) #取得数据量百分之八十的位置
    training_data = data[:offset] #读取前offset个数据
     #对数据进行归一化处理
    maximums,minimums,avgs=\
            training_data.max(axis=0),\
            training_data.min(axis=0),\
    training_data.sum(axis=0)/training_data.shape[0]
    for i in range(feature_num):
        data[:,i] = (data[:,i] - avgs[i]) / (maximums[i] - minimums[i])
        
    
    train_data = data[:offset]#读取前offset个数据
    test_data = data[offset:]#读取剩下的数据
    return train_data,test_data

In [2]:
import torch
train_data,test_data = load_data() #读取文件函数，上面已经定义了
x = train_data[:,:-1] #把数据的前13个参数给x，房子的信息参数
y = train_data[:,-1:] #把最后一个参数给y，真实房价
#torch方法做
a = torch.from_numpy(x) #用torch来做，这一步是转换数据类型，把numpy的array转换成torch的tenser
b = torch.from_numpy(y)
a.requires_grad = True #设置a和b的grad，用来计算backforword
b.requires_grad = True

In [3]:
w = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,-0.1,-0.2,-0.3,-0.4,0.0] #测试
w = np.array(w).reshape([13,1])

In [4]:
class Network(object):
    def __init__(self, num_of_weights): #num_of_weights-参数的维度
        #随机产生w的初始值
        #为了保持程序每次运行的一致性，
        #此处设置固定的随机数种子
        np.random.seed(0) #设置随机数种子
        self.w = np.random.randn(num_of_weights, 1) #声明有13个参数的随机的参数表
        self.b = 0. #声明附加参数
        
    def forward(self, x): #x - 训练数据，维度和num_of_weights一致
        z = np.dot(x, self.w) + self.b #计算forward—x.dot(self.w)—用x矩阵和参数w矩阵进行矩阵乘法运算 再加上附加参数
        return z
    def loss(self,z,y): #计算loss，z-计算出的forword数据，y-真实数据
        error = z-y #loss减去真实数据
        cost = np.square(error) #平方
        cost = np.mean(cost)  #取平均数
        return cost
    def gradient(self,x,y): #计算参数的grad，x-训练数据，维度和num_of_weights一致,y-真实数据
        z = self.forward(x) #forword计算
        gradient_w = (z-y)*x #反向求导
        gradient_w = np.mean(gradient_w, axis = 0)#压缩列，取所有列的平均值
        gradient_w = gradient_w[:,np.newaxis] #重新设置成二维矩阵，不然无法进行操作
        gradient_b = (z-y) #求附加参数的偏导
        gradient_b = np.mean(gradient_b) #取平均值
        
        return gradient_w, gradient_b #返回求出的w_grad和b_grad,
    def update(self, gradient_w, gradient_b, eta = 0.01): #更新参数，gradient_w参数w的变化率，gradient_b-参数b的变化率
        self.w = self.w - eta * gradient_w #用原来的参数w减去w_grad，用以更新参数w
        self.b = self.b - eta * gradient_b #用原来的参数b减去b_grad，用以更新参数b
    def train(self, x, y, iterations = 100, eta = 0.01): #训练函数 iterations-训练次数 eta-参数改变幅度
        losses = [] #losses表，用于记录所有训练的loss值
        for i in range(iterations):
            z = self.forward(x) #计算forword
            L = self.loss(z,y) #计算loss
            gradient_w, gradient_b = self.gradient(x, y) #计算w_grad和b_grad
            self.update(gradient_w, gradient_b, eta) #用w_grad和b_grad更新参数w和参数b
            losses.append(L) #将loss加入loss表
#             if(i + 1)%10 == 0:#每十次训练查看一次loss
#                 print('iter{},loss{}'.format(i,L))
        return losses #返回losses表
    
    
#torch方法做


In [5]:
import torch.nn as nn
class Network1(torch.nn.Module):
    def __init__(self, num_of_weights):
        torch.manual_seed(0)
        
        self.a = torch.randn(num_of_weights, 1,dtype=torch.float64,requires_grad = True)
        self.b = 0.
        
#         super(Network1, self).__init__()
#         torch.manual_seed(0)
# #         self.linear1 = torch.randn(num_of_weights,1, bias = False)
# #         self.linear2 = 0.
#         self.linear1 = torch.nn.Linear(num_of_weights,1, bias = False)
#         self.linear2 = 0.

    def forward(self, x):
        z = x.mm(self.a)
        return z

In [1]:
# net = Network(13)
# x1 = x[:3]
# y1 = y[:3]
# z = net.forward(x1)
# print(z)
# print(y1)
# loss = net.loss(z,y1)
# # print(loss)


# gradient_w = 2 *(z - y1)* x1
# print('gradient_w', gradient_w.shape)
# gradient_w = np.mean(gradient_w, axis = 0)
# gradient_w = gradient_w[:,np.newaxis]
# print('gradient_w', gradient_w.shape)
# print(gradient_w)

# gradient_b = (z-y1)
# gradient_b = np.mean(gradient_b)

# gradient_b


# eta = 0.1
# net.w[5] = net.w[5]-eta* gradient_w5
# net.w[9] = net.w[9]-eta* gradient_w9
#计算Z和loss



# gradient_w5 = gradient_w[5][0]
# gradient_w9 = gradient_w[9][0]
# print(net.w[5][0],net.w[9][0], loss)
# print(gradient_w5,gradient_w9)

# [ -0.14407246   0.25288296  -1.91658252 ...  -1.94680358   0.34845553
#    -1.1807961 ]
import torch
import matplotlib.pyplot as plt
train_data,test_data = load_data() #读取文件函数，上面已经定义了
x = train_data[:,:-1] #把数据的前13个参数给x，房子的信息参数
y = train_data[:,-1:] #把最后一个参数给y，真实房价
#torch方法做
a = torch.from_numpy(x) #用torch来做，这一步是转换数据类型，把numpy的array转换成torch的tenser
b = torch.from_numpy(y)
a.requires_grad = True #设置a和b的grad，用来计算backforword
b.requires_grad = True

net = Network(13)
num_iterations = 10000
#训练
losses = net.train(x, y, iterations = num_iterations, eta = 0.01)
#画出图表
plot_x = np.arange(num_iterations)
plot_y = np.array(losses)
plt.plot(plot_x,plot_y)
plt.show()

x = test_data[:,:-1]
y = test_data[:,-1:]
z = net.forward(x)
L = net.loss(z,y)
print(L)

ModuleNotFoundError: No module named 'matplotlib'